# PRUEBA DE CONOCIMIENTO - USO Y ANALISIS DE DATOS CON PYTHON
## Parte 1: Validaciones y base de datos
Dataset: Empaquetamiento de servicios fijos (Postdata)

## Configuracion inicial

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuracion
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

## Carga de datos

In [2]:
print("="*80)
print("CARGANDO DATOS...")
print("="*80)

url = "https://www.postdata.gov.co/sites/default/files/datasets/data/EMPAQUETAMIENTO_FIJO_11.csv"
df = pd.read_csv(url, sep=None, engine='python', on_bad_lines='skip', encoding='utf-8')
df.columns = df.columns.str.replace('\ufeff', '')

print(f"Datos cargados exitosamente")
print(f"  - Registros totales: {len(df):,}")
print(f"  - Columnas: {len(df.columns)}")
print(f"  - Rango de anos: {df['ANNO'].min()} - {df['ANNO'].max()}")

CARGANDO DATOS...
Datos cargados exitosamente
  - Registros totales: 2,989,763
  - Columnas: 21
  - Rango de anos: 2022 - 2025


## Filtrado de datos: Anos 2023 y 2024

In [3]:
print("\n" + "="*80)
print("FILTRADO DE DATOS: ANOS 2023 Y 2024")
print("="*80)

df_analisis = df[df['ANNO'].isin([2023, 2024])].copy()
print(f"Registros 2023-2024: {len(df_analisis):,}")
print(f"  - Registros 2023: {len(df_analisis[df_analisis['ANNO']==2023]):,}")
print(f"  - Registros 2024: {len(df_analisis[df_analisis['ANNO']==2024]):,}")


FILTRADO DE DATOS: ANOS 2023 Y 2024
Registros 2023-2024: 1,750,659
  - Registros 2023: 898,318
  - Registros 2024: 852,341


## ACTIVIDAD 1: VALIDACION DE ESTRUCTURA

In [4]:
print("\n" + "="*80)
print("ACTIVIDAD 1: VALIDACION DE ESTRUCTURA")
print("="*80)

campos_esperados = {
    'ANNO': 'int64', 'TRIMESTRE': 'int64', 'ID_EMPRESA': 'int64', 'EMPRESA': 'object',
    'ID_DEPARTAMENTO': 'int64', 'DEPARTAMENTO': 'object', 'ID_MUNICIPIO': 'int64',
    'MUNICIPIO': 'object', 'ID_SEGMENTO': 'int64', 'SEGMENTO': 'object',
    'ID_SERVICIO_PAQUETE': 'int64', 'SERVICIO_PAQUETE': 'object',
    'VELOCIDAD_EFECTIVA_DOWNSTREAM': 'float64', 'VELOCIDAD_EFECTIVA_UPSTREAM': 'float64',
    'ID_TECNOLOGIA_ACCESO': 'int64', 'TECNOLOGIA': 'object', 'ID_ESTADO': 'int64',
    'ESTADO': 'object', 'CANTIDAD_LINEAS_ACCESOS': 'int64',
    'VALOR_FACTURADO_O_COBRADO': 'int64', 'OTROS_VALORES_FACTURADOS': 'int64'
}


ACTIVIDAD 1: VALIDACION DE ESTRUCTURA


### 1.1 Validacion de campos

In [5]:
print("\n1.1 Validacion de campos:")
campos_presentes = set(df_analisis.columns)
campos_esperados_set = set(campos_esperados.keys())
if campos_esperados_set == campos_presentes:
    print("  Todos los campos obligatorios estan presentes")
else:
    print(f"  Diferencias encontradas")


1.1 Validacion de campos:
  Todos los campos obligatorios estan presentes


### 1.2 Tipos de datos actuales vs esperados

In [6]:
print("\n1.2 Tipos de datos actuales vs esperados:")
print("-" * 80)
conversiones_necesarias = {}
for campo in sorted(campos_esperados.keys()):
    if campo in df_analisis.columns:
        tipo_actual = str(df_analisis[campo].dtype)
        tipo_esperado = campos_esperados[campo]
        if tipo_actual == tipo_esperado:
            match = "OK"
        elif tipo_actual == 'object' and tipo_esperado in ['int64', 'float64']:
            match = "CONVERTIR"
            conversiones_necesarias[campo] = tipo_esperado
        else:
            match = "TIPO DIFERENTE"
        print(f"  {match:25s} {campo:35s} | Actual: {tipo_actual:10s} | Esperado: {tipo_esperado}")


1.2 Tipos de datos actuales vs esperados:
--------------------------------------------------------------------------------
  OK                        ANNO                                | Actual: int64      | Esperado: int64
  OK                        CANTIDAD_LINEAS_ACCESOS             | Actual: int64      | Esperado: int64
  OK                        DEPARTAMENTO                        | Actual: object     | Esperado: object
  OK                        EMPRESA                             | Actual: object     | Esperado: object
  OK                        ESTADO                              | Actual: object     | Esperado: object
  OK                        ID_DEPARTAMENTO                     | Actual: int64      | Esperado: int64
  OK                        ID_EMPRESA                          | Actual: int64      | Esperado: int64
  OK                        ID_ESTADO                           | Actual: int64      | Esperado: int64
  OK                        ID_MUNICIPIO         

### 1.3 Conversion de tipos de datos

In [7]:
if conversiones_necesarias:
    print("\n1.3 Convirtiendo tipos de datos:")
    print("-" * 80)
    for campo, tipo_esperado in conversiones_necesarias.items():
        try:
            if tipo_esperado == 'int64':
                df_analisis[campo] = pd.to_numeric(df_analisis[campo], errors='coerce').astype('Int64')
                print(f"  {campo} convertido a int64")
            elif tipo_esperado == 'float64':
                df_analisis[campo] = pd.to_numeric(df_analisis[campo], errors='coerce')
                print(f"  {campo} convertido a float64")
        except Exception as e:
            print(f"  Error convirtiendo {campo}: {e}")


1.3 Convirtiendo tipos de datos:
--------------------------------------------------------------------------------
  OTROS_VALORES_FACTURADOS convertido a int64
  VALOR_FACTURADO_O_COBRADO convertido a int64
  VELOCIDAD_EFECTIVA_DOWNSTREAM convertido a float64
  VELOCIDAD_EFECTIVA_UPSTREAM convertido a float64


### 1.4 Validacion de rangos categoricos

In [8]:
print("\n1.4 Validacion de rangos categoricos:")
print("-" * 80)

# Validar rangos
validaciones = {
    'ID_SERVICIO_PAQUETE': (set(range(1, 8)), "1-7"),
    'ID_ESTADO': ({1, 2}, "1-2"),
    'TRIMESTRE': ({1, 2, 3, 4}, "1-4")
}

for campo, (valores_validos, rango_str) in validaciones.items():
    valores_encontrados = set(df_analisis[campo].dropna().unique())
    invalidos = valores_encontrados - valores_validos
    if not invalidos:
        print(f"  {campo}: Todos en rango {rango_str}")
    else:
        print(f"  {campo}: Valores fuera de rango: {invalidos}")


1.4 Validacion de rangos categoricos:
--------------------------------------------------------------------------------
  ID_SERVICIO_PAQUETE: Todos en rango 1-7
  ID_ESTADO: Todos en rango 1-2
  TRIMESTRE: Todos en rango 1-4


### 1.5 Analisis de valores unicos en variables categoricas

In [9]:
print("\n1.5 Analisis de valores unicos en variables categoricas:")
print("="*80)

# ANNO
print("\nANNO:")
print(f"  Total valores unicos: {df_analisis['ANNO'].nunique()}")
for val, cnt in df_analisis['ANNO'].value_counts().sort_index().items():
    pct = cnt/len(df_analisis)*100
    print(f"    {val}: {cnt:8,} ({pct:5.2f}%)")

# TRIMESTRE
print("\nTRIMESTRE:")
print(f"  Total valores unicos: {df_analisis['TRIMESTRE'].nunique()}")
for val, cnt in df_analisis['TRIMESTRE'].value_counts().sort_index().items():
    pct = cnt/len(df_analisis)*100
    print(f"    Trimestre {val}: {cnt:8,} ({pct:5.2f}%)")

# ESTADO
print("\nESTADO:")
print(f"  Total valores unicos: {df_analisis['ESTADO'].nunique()}")
for val, cnt in df_analisis['ESTADO'].value_counts().items():
    pct = cnt/len(df_analisis)*100
    print(f"    {val}: {cnt:8,} ({pct:5.2f}%)")

# SEGMENTO
print("\nSEGMENTO:")
print(f"  Total valores unicos: {df_analisis['SEGMENTO'].nunique()}")
segmentos = df_analisis.groupby(['ID_SEGMENTO', 'SEGMENTO']).size().reset_index(name='count').sort_values('ID_SEGMENTO')
for _, row in segmentos.iterrows():
    pct = row['count']/len(df_analisis)*100
    print(f"    [{row['ID_SEGMENTO']}] {row['SEGMENTO']:45s}: {row['count']:8,} ({pct:5.2f}%)")

# SERVICIO_PAQUETE
print("\nSERVICIO_PAQUETE:")
print(f"  Total valores unicos: {df_analisis['SERVICIO_PAQUETE'].nunique()}")
servicios = df_analisis.groupby(['ID_SERVICIO_PAQUETE', 'SERVICIO_PAQUETE']).size().reset_index(name='count').sort_values('ID_SERVICIO_PAQUETE')
for _, row in servicios.iterrows():
    pct = row['count']/len(df_analisis)*100
    print(f"    [{row['ID_SERVICIO_PAQUETE']}] {row['SERVICIO_PAQUETE']:50s}: {row['count']:8,} ({pct:5.2f}%)")

# TECNOLOGIA
print("\nTECNOLOGIA:")
n_tec = df_analisis['TECNOLOGIA'].nunique()
print(f"  Total valores unicos: {n_tec}")
tecnologias = df_analisis.groupby(['ID_TECNOLOGIA_ACCESO', 'TECNOLOGIA']).size().reset_index(name='count').sort_values('count', ascending=False)
if n_tec <= 20:
    for _, row in tecnologias.iterrows():
        pct = row['count']/len(df_analisis)*100
        print(f"    [{row['ID_TECNOLOGIA_ACCESO']:2d}] {row['TECNOLOGIA']:45s}: {row['count']:8,} ({pct:5.2f}%)")
else:
    for _, row in tecnologias.head(15).iterrows():
        pct = row['count']/len(df_analisis)*100
        print(f"    [{row['ID_TECNOLOGIA_ACCESO']:2d}] {row['TECNOLOGIA']:45s}: {row['count']:8,} ({pct:5.2f}%)")

# DEPARTAMENTO
print("\nDEPARTAMENTO:")
n_dept = df_analisis['DEPARTAMENTO'].nunique()
print(f"  Total valores unicos: {n_dept}")
departamentos = df_analisis.groupby(['ID_DEPARTAMENTO', 'DEPARTAMENTO']).size().reset_index(name='count').sort_values('count', ascending=False)
if n_dept <= 40:
    for _, row in departamentos.iterrows():
        pct = row['count']/len(df_analisis)*100
        print(f"    [{row['ID_DEPARTAMENTO']:2d}] {row['DEPARTAMENTO']:30s}: {row['count']:8,} ({pct:5.2f}%)")
else:
    for _, row in departamentos.head(15).iterrows():
        pct = row['count']/len(df_analisis)*100
        print(f"    [{row['ID_DEPARTAMENTO']:2d}] {row['DEPARTAMENTO']:30s}: {row['count']:8,} ({pct:5.2f}%)")

# MUNICIPIO Y EMPRESA (solo conteo)
print("\nMUNICIPIO:")
print(f"  Total valores unicos: {df_analisis['MUNICIPIO'].nunique()}")
print("\nEMPRESA:")
print(f"  Total valores unicos: {df_analisis['EMPRESA'].nunique()}")


1.5 Analisis de valores unicos en variables categoricas:

ANNO:
  Total valores unicos: 2
    2023:  898,318 (51.31%)
    2024:  852,341 (48.69%)

TRIMESTRE:
  Total valores unicos: 4
    Trimestre 1:  439,737 (25.12%)
    Trimestre 2:  438,224 (25.03%)
    Trimestre 3:  447,378 (25.55%)
    Trimestre 4:  425,320 (24.29%)

ESTADO:
  Total valores unicos: 2
    Activo en funcionamiento: 1,487,030 (84.94%)
    Suspensión temporal:  263,629 (15.06%)

SEGMENTO:
  Total valores unicos: 10
    [101] Residencial - Estrato 1                      :  280,251 (16.01%)
    [102] Residencial - Estrato 2                      :  414,245 (23.66%)
    [103] Residencial - Estrato 3                      :  290,140 (16.57%)
    [104] Residencial - Estrato 4                      :  147,517 ( 8.43%)
    [105] Residencial - Estrato 5                      :   64,070 ( 3.66%)
    [106] Residencial - Estrato 6                      :   50,453 ( 2.88%)
    [107] Corporativo                                  :  45

## ACTIVIDAD 2: ANALISIS DE VALORES NULOS Y ERRORES

In [10]:
print("\n" + "="*80)
print("ACTIVIDAD 2: ANALISIS DE VALORES NULOS Y ERRORES")
print("="*80)


ACTIVIDAD 2: ANALISIS DE VALORES NULOS Y ERRORES


### 2.1 Campos con valores nulos

In [11]:
print("\n2.1 Campos con valores nulos:")
print("-" * 80)
nulos = df_analisis.isnull().sum()
nulos_pct = (nulos / len(df_analisis) * 100).round(2)
df_nulos = pd.DataFrame({'Campo': nulos.index, 'Nulos': nulos.values, 'Porcentaje': nulos_pct.values})
df_nulos = df_nulos[df_nulos['Nulos'] > 0].sort_values('Nulos', ascending=False)
if len(df_nulos) > 0:
    print(df_nulos.to_string(index=False))
else:
    print("  No se encontraron valores nulos")


2.1 Campos con valores nulos:
--------------------------------------------------------------------------------
                        Campo  Nulos  Porcentaje
  VELOCIDAD_EFECTIVA_UPSTREAM  69974        4.00
VELOCIDAD_EFECTIVA_DOWNSTREAM  23768        1.36
    VALOR_FACTURADO_O_COBRADO   1354        0.08
     OTROS_VALORES_FACTURADOS     17        0.00


### 2.2 Analisis de nulos por contexto

In [12]:
print("\n2.2 Analisis de nulos por contexto:")
print("-" * 80)

# Velocidades: verificar si son servicios que requieren internet
servicios_con_internet = [1, 4, 5, 7]  # Internet, Duo Play 1, Duo Play 2, Triple Play
print("  VELOCIDAD_EFECTIVA_DOWNSTREAM:")
for servicio in sorted(df_analisis['ID_SERVICIO_PAQUETE'].unique()):
    nombre = df_analisis[df_analisis['ID_SERVICIO_PAQUETE']==servicio]['SERVICIO_PAQUETE'].iloc[0]
    nulos_serv = df_analisis[(df_analisis['ID_SERVICIO_PAQUETE']==servicio) & 
                             (df_analisis['VELOCIDAD_EFECTIVA_DOWNSTREAM'].isna())].shape[0]
    total_serv = df_analisis[df_analisis['ID_SERVICIO_PAQUETE']==servicio].shape[0]
    pct_serv = (nulos_serv/total_serv*100) if total_serv > 0 else 0
    requiere = "SI" if servicio in servicios_con_internet else "NO"
    print(f"    [{servicio}] {nombre[:35]:35s} - Requiere Internet: {requiere} | Nulos: {nulos_serv:5,} ({pct_serv:5.2f}%)")

print("\n  VELOCIDAD_EFECTIVA_UPSTREAM:")
for servicio in sorted(df_analisis['ID_SERVICIO_PAQUETE'].unique()):
    nombre = df_analisis[df_analisis['ID_SERVICIO_PAQUETE']==servicio]['SERVICIO_PAQUETE'].iloc[0]
    nulos_serv = df_analisis[(df_analisis['ID_SERVICIO_PAQUETE']==servicio) & 
                             (df_analisis['VELOCIDAD_EFECTIVA_UPSTREAM'].isna())].shape[0]
    total_serv = df_analisis[df_analisis['ID_SERVICIO_PAQUETE']==servicio].shape[0]
    pct_serv = (nulos_serv/total_serv*100) if total_serv > 0 else 0
    requiere = "SI" if servicio in servicios_con_internet else "NO"
    print(f"    [{servicio}] {nombre[:35]:35s} - Requiere Internet: {requiere} | Nulos: {nulos_serv:5,} ({pct_serv:5.2f}%)")

print("\n  VALORES FACTURADOS:")
print(f"    VALOR_FACTURADO_O_COBRADO - Nulos: {df_analisis['VALOR_FACTURADO_O_COBRADO'].isna().sum():,}")
print(f"    OTROS_VALORES_FACTURADOS  - Nulos: {df_analisis['OTROS_VALORES_FACTURADOS'].isna().sum():,}")


2.2 Analisis de nulos por contexto:
--------------------------------------------------------------------------------
  VELOCIDAD_EFECTIVA_DOWNSTREAM:
    [1] Internet fijo                       - Requiere Internet: SI | Nulos: 23,275 ( 3.17%)
    [2] Telefonía fija                      - Requiere Internet: NO | Nulos:     0 ( 0.00%)
    [3] Televisión por suscripción          - Requiere Internet: NO | Nulos:     0 ( 0.00%)
    [4] Duo Play 1 (Telefonía fija + Intern - Requiere Internet: SI | Nulos:   467 ( 0.16%)
    [5] Duo Play 2 (Internet fijo y TV por  - Requiere Internet: SI | Nulos:    16 ( 0.01%)
    [6] Duo Play 3 (Telefonía fija y TV por - Requiere Internet: NO | Nulos:     0 ( 0.00%)
    [7] Triple Play (Telefonía fija + Inter - Requiere Internet: SI | Nulos:    10 ( 0.00%)

  VELOCIDAD_EFECTIVA_UPSTREAM:
    [1] Internet fijo                       - Requiere Internet: SI | Nulos: 55,349 ( 7.53%)
    [2] Telefonía fija                      - Requiere Internet: NO | Nulos:   

### 2.3 Validacion de cobertura geografica

In [13]:
print("\n2.3 Validacion de cobertura geografica:")
print("-" * 80)
print("\nTop 10 departamentos:")
for val, cnt in df_analisis['DEPARTAMENTO'].value_counts().head(10).items():
    pct = cnt/len(df_analisis)*100
    n_mun = df_analisis[df_analisis['DEPARTAMENTO']==val]['MUNICIPIO'].nunique()
    n_emp = df_analisis[df_analisis['DEPARTAMENTO']==val]['EMPRESA'].nunique()
    print(f"  {val:30s}: {cnt:8,} ({pct:5.2f}%) | {n_mun:3d} municipios | {n_emp:3d} empresas")

print("\nTop 10 empresas:")
for val, cnt in df_analisis['EMPRESA'].value_counts().head(10).items():
    pct = cnt/len(df_analisis)*100
    n_dept = df_analisis[df_analisis['EMPRESA']==val]['DEPARTAMENTO'].nunique()
    print(f"  {val[:50]:50s}: {cnt:8,} ({pct:5.2f}%) | {n_dept:2d} departamentos")


2.3 Validacion de cobertura geografica:
--------------------------------------------------------------------------------

Top 10 departamentos:
  ANTIOQUIA                     :  268,970 (15.36%) | 125 municipios | 273 empresas
  CUNDINAMARCA                  :  234,624 (13.40%) | 116 municipios | 237 empresas
  VALLE DEL CAUCA               :  172,413 ( 9.85%) |  42 municipios | 241 empresas
  SANTANDER                     :  108,131 ( 6.18%) |  87 municipios | 165 empresas
  BOYACÁ                        :   87,465 ( 5.00%) | 123 municipios | 135 empresas
  CALDAS                        :   69,606 ( 3.98%) |  27 municipios | 106 empresas
  ATLÁNTICO                     :   63,960 ( 3.65%) |  23 municipios |  89 empresas
  TOLIMA                        :   62,564 ( 3.57%) |  47 municipios | 141 empresas
  RISARALDA                     :   60,179 ( 3.44%) |  14 municipios |  82 empresas
  NORTE DE SANTANDER            :   56,743 ( 3.24%) |  40 municipios |  87 empresas

Top 10 empresa

## ACTIVIDAD 3: CONSISTENCIA CANTIDAD LINEAS vs VALOR FACTURADO

In [14]:
print("\n" + "="*80)
print("ACTIVIDAD 3: CONSISTENCIA CANTIDAD LINEAS vs VALOR FACTURADO")
print("="*80)


ACTIVIDAD 3: CONSISTENCIA CANTIDAD LINEAS vs VALOR FACTURADO


### 3.1 Verificacion de casos inconsistentes

In [15]:
print("\n3.1 Verificacion de casos inconsistentes:")
print("-" * 80)

# Caso 1: Lineas > 0 pero valor = 0 (servicio sin facturacion)
caso1 = df_analisis[(df_analisis['CANTIDAD_LINEAS_ACCESOS'] > 0) & 
                     (df_analisis['VALOR_FACTURADO_O_COBRADO'] == 0)]
print(f"  Lineas > 0 pero valor = 0: {len(caso1):,} ({len(caso1)/len(df_analisis)*100:.2f}%)")
print(f"    Posibles causas: servicios gratuitos, promocionales, uso interno")

# Caso 2: Lineas = 0 pero valor > 0 (facturacion sin lineas activas)
caso2 = df_analisis[(df_analisis['CANTIDAD_LINEAS_ACCESOS'] == 0) & 
                     (df_analisis['VALOR_FACTURADO_O_COBRADO'] > 0)]
print(f"  Lineas = 0 pero valor > 0: {len(caso2):,} ({len(caso2)/len(df_analisis)*100:.2f}%)")
print(f"    Posibles causas: errores de registro, cobros pendientes")

# Caso 3: Ambos en cero
caso3 = df_analisis[(df_analisis['CANTIDAD_LINEAS_ACCESOS'] == 0) & 
                     (df_analisis['VALOR_FACTURADO_O_COBRADO'] == 0)]
print(f"  Lineas = 0 y valor = 0: {len(caso3):,} ({len(caso3)/len(df_analisis)*100:.2f}%)")
print(f"    Posibles causas: registros sin actividad")

# Caso 4: Consistentes (ambos > 0)
caso4 = df_analisis[(df_analisis['CANTIDAD_LINEAS_ACCESOS'] > 0) & 
                     (df_analisis['VALOR_FACTURADO_O_COBRADO'] > 0)]
print(f"  Registros consistentes: {len(caso4):,} ({len(caso4)/len(df_analisis)*100:.2f}%)")


3.1 Verificacion de casos inconsistentes:
--------------------------------------------------------------------------------
  Lineas > 0 pero valor = 0: 104,364 (5.96%)
    Posibles causas: servicios gratuitos, promocionales, uso interno
  Lineas = 0 pero valor > 0: 13,011 (0.74%)
    Posibles causas: errores de registro, cobros pendientes
  Lineas = 0 y valor = 0: 2,217 (0.13%)
    Posibles causas: registros sin actividad
  Registros consistentes: 1,629,713 (93.09%)


### 3.2 Estadisticas de valor por linea

In [16]:
print("\n3.2 Estadisticas de valor por linea (solo registros consistentes):")
print("-" * 80)
df_analisis['VALOR_POR_LINEA'] = np.where(
    df_analisis['CANTIDAD_LINEAS_ACCESOS'] > 0,
    df_analisis['VALOR_FACTURADO_O_COBRADO'] / df_analisis['CANTIDAD_LINEAS_ACCESOS'],
    np.nan
)

stats = df_analisis[df_analisis['VALOR_POR_LINEA'] > 0]['VALOR_POR_LINEA'].describe()
print(f"  Media: ${stats['mean']:,.0f} COP")
print(f"  Mediana: ${stats['50%']:,.0f} COP")
print(f"  Q1 (25%): ${stats['25%']:,.0f} COP")
print(f"  Q3 (75%): ${stats['75%']:,.0f} COP")
print(f"  Minimo: ${stats['min']:,.0f} COP")
print(f"  Maximo: ${stats['max']:,.0f} COP")


3.2 Estadisticas de valor por linea (solo registros consistentes):
--------------------------------------------------------------------------------
  Media: $784,483 COP
  Mediana: $192,857 COP
  Q1 (25%): $105,042 COP
  Q3 (75%): $313,260 COP
  Minimo: $0 COP
  Maximo: $2,166,075,866 COP


### 3.3 Valores atipicos (outliers)

In [17]:
print("\n3.3 Valores atipicos (outliers):")
print("-" * 80)
Q1 = stats['25%']
Q3 = stats['75%']
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

outliers_bajos = df_analisis[(df_analisis['VALOR_POR_LINEA'] > 0) & 
                              (df_analisis['VALOR_POR_LINEA'] < limite_inferior)]
outliers_altos = df_analisis[df_analisis['VALOR_POR_LINEA'] > limite_superior]

print(f"  Valores atipicos bajos (< ${limite_inferior:,.0f}): {len(outliers_bajos):,} ({len(outliers_bajos)/len(df_analisis)*100:.2f}%)")
print(f"  Valores atipicos altos (> ${limite_superior:,.0f}): {len(outliers_altos):,} ({len(outliers_altos)/len(df_analisis)*100:.2f}%)")


3.3 Valores atipicos (outliers):
--------------------------------------------------------------------------------
  Valores atipicos bajos (< $-207,285): 0 (0.00%)
  Valores atipicos altos (> $625,587): 178,302 (10.18%)


### 3.4 Analisis por tipo de servicio

In [18]:
print("\n3.4 Analisis por tipo de servicio:")
print("-" * 80)
for servicio in sorted(df_analisis['ID_SERVICIO_PAQUETE'].unique()):
    df_serv = df_analisis[(df_analisis['ID_SERVICIO_PAQUETE'] == servicio) & 
                          (df_analisis['VALOR_POR_LINEA'] > 0)]
    if len(df_serv) > 0:
        nombre = df_serv['SERVICIO_PAQUETE'].iloc[0]
        mediana = df_serv['VALOR_POR_LINEA'].median()
        print(f"  [{servicio}] {nombre[:45]:45s}: ${mediana:>10,.0f} COP (mediana)")


3.4 Analisis por tipo de servicio:
--------------------------------------------------------------------------------
  [1] Internet fijo                                : $   225,000 COP (mediana)
  [2] Telefonía fija                               : $    55,201 COP (mediana)
  [3] Televisión por suscripción                   : $   139,687 COP (mediana)
  [4] Duo Play 1 (Telefonía fija + Internet fijo)  : $   158,250 COP (mediana)
  [5] Duo Play 2 (Internet fijo y TV por suscripció: $   225,232 COP (mediana)
  [6] Duo Play 3 (Telefonía fija y TV por suscripci: $   110,917 COP (mediana)
  [7] Triple Play (Telefonía fija + Internet fijo +: $   195,617 COP (mediana)


## ACTIVIDAD 4: VERIFICACION DE DUPLICADOS

In [19]:
print("\n" + "="*80)
print("ACTIVIDAD 4: VERIFICACION DE DUPLICADOS")
print("="*80)


ACTIVIDAD 4: VERIFICACION DE DUPLICADOS


### 4.1 Duplicados completos (todos los campos identicos)

In [20]:
print("\n4.1 INTERPRETACION 1: Duplicados completos (todos los campos identicos)")
print("-" * 80)
duplicados_completos = df_analisis[df_analisis.duplicated(keep=False)]
print(f"  Registros duplicados: {len(duplicados_completos):,} ({len(duplicados_completos)/len(df_analisis)*100:.2f}%)")
if len(duplicados_completos) > 0:
    # Contar grupos unicos de duplicados
    num_grupos = df_analisis[df_analisis.duplicated(keep='first')].shape[0]
    if num_grupos > 0:
        print(f"  Grupos de duplicados: {num_grupos:,}")
        print(f"  Promedio de duplicados por grupo: {len(duplicados_completos) / num_grupos:.2f}")
    else:
        print(f"  Nota: Todos los duplicados aparecen exactamente 2 veces")
        print(f"  Grupos de duplicados: {len(duplicados_completos)//2:,}")


4.1 INTERPRETACION 1: Duplicados completos (todos los campos identicos)
--------------------------------------------------------------------------------
  Registros duplicados: 900 (0.05%)
  Grupos de duplicados: 533
  Promedio de duplicados por grupo: 1.69


### 4.2 Duplicados por combinacion de dimensiones

In [21]:
print("\n4.2 INTERPRETACION 2: Duplicados por combinacion de dimensiones")
print("-" * 80)
print("  (Municipio-departamento + Segmento + Servicio + Ano-trimestre)")

claves_combinadas = ['ANNO', 'TRIMESTRE', 'ID_MUNICIPIO', 'ID_DEPARTAMENTO', 
                     'ID_SEGMENTO', 'ID_SERVICIO_PAQUETE']
duplicados_combinados = df_analisis[df_analisis.duplicated(subset=claves_combinadas, keep=False)]
print(f"  Registros duplicados: {len(duplicados_combinados):,} ({len(duplicados_combinados)/len(df_analisis)*100:.2f}%)")

if len(duplicados_combinados) > 0:
    grupos_combinados = duplicados_combinados.groupby(claves_combinadas).size()
    print(f"  Grupos de duplicados: {len(grupos_combinados):,}")
    print(f"  Promedio de registros por grupo: {len(duplicados_combinados) / len(grupos_combinados):.2f}")
    
    print("\n  Que diferencia a estos registros 'duplicados'?")
    ejemplo = duplicados_combinados.groupby(claves_combinadas).head(100)
    campos_variantes = []
    
    for col in df_analisis.columns:
        if col not in claves_combinadas:
            variacion = ejemplo.groupby(claves_combinadas)[col].nunique()
            if (variacion > 1).any():
                campos_variantes.append(col)
    
    if campos_variantes:
        print("  Campos que varian dentro de los grupos:")
        for campo in campos_variantes[:10]:
            unicos_prom = duplicados_combinados.groupby(claves_combinadas)[campo].nunique().mean()
            print(f"    - {campo}: {unicos_prom:.2f} valores unicos promedio")


4.2 INTERPRETACION 2: Duplicados por combinacion de dimensiones
--------------------------------------------------------------------------------
  (Municipio-departamento + Segmento + Servicio + Ano-trimestre)
  Registros duplicados: 1,698,761 (97.04%)
  Grupos de duplicados: 111,360
  Promedio de registros por grupo: 15.25

  Que diferencia a estos registros 'duplicados'?
  Campos que varian dentro de los grupos:
    - ID_EMPRESA: 2.88 valores unicos promedio
    - EMPRESA: 2.88 valores unicos promedio
    - VELOCIDAD_EFECTIVA_DOWNSTREAM: 7.68 valores unicos promedio
    - VELOCIDAD_EFECTIVA_UPSTREAM: 6.62 valores unicos promedio
    - ID_TECNOLOGIA_ACCESO: 2.02 valores unicos promedio
    - TECNOLOGIA: 2.02 valores unicos promedio
    - ID_ESTADO: 1.49 valores unicos promedio
    - ESTADO: 1.49 valores unicos promedio
    - CANTIDAD_LINEAS_ACCESOS: 7.09 valores unicos promedio
    - VALOR_FACTURADO_O_COBRADO: 14.45 valores unicos promedio


### 4.3 Duplicados por cada dimension individual

In [22]:
print("\n4.3 INTERPRETACION 3: Duplicados por cada dimension individual")
print("-" * 80)

dimensiones = {
    'Municipio-Departamento': ['ID_MUNICIPIO', 'ID_DEPARTAMENTO'],
    'Segmento': ['ID_SEGMENTO'],
    'Servicio/Paquete': ['ID_SERVICIO_PAQUETE'],
    'Ano-Trimestre': ['ANNO', 'TRIMESTRE']
}

for nombre, campos in dimensiones.items():
    dup = df_analisis[df_analisis.duplicated(subset=campos, keep=False)]
    combinaciones = df_analisis[campos].drop_duplicates().shape[0]
    print(f"  {nombre:25s}: {len(dup):8,} duplicados ({len(dup)/len(df_analisis)*100:5.2f}%) | {combinaciones:,} combinaciones unicas")


4.3 INTERPRETACION 3: Duplicados por cada dimension individual
--------------------------------------------------------------------------------
  Municipio-Departamento   : 1,750,659 duplicados (100.00%) | 1,122 combinaciones unicas
  Segmento                 : 1,750,659 duplicados (100.00%) | 10 combinaciones unicas
  Servicio/Paquete         : 1,750,659 duplicados (100.00%) | 7 combinaciones unicas
  Ano-Trimestre            : 1,750,659 duplicados (100.00%) | 8 combinaciones unicas


## SECCION 5: DECISIONES DE LIMPIEZA

In [23]:
print("\n" + "="*80)
print("DECISIONES DE LIMPIEZA DE DATOS")
print("="*80)


DECISIONES DE LIMPIEZA DE DATOS


### 5.1 Correccion del campo TECNOLOGIA

In [24]:
print("\n5.1 Correccion del campo TECNOLOGIA:")
print("-" * 80)
print("  Estandarizando 'NA (No Aplica)' -> 'NA'")
df_analisis['TECNOLOGIA'] = df_analisis['TECNOLOGIA'].replace('NA (No Aplica)', 'NA')
na_count = (df_analisis['TECNOLOGIA'] == 'NA').sum()
print(f"  Total registros con TECNOLOGIA='NA': {na_count:,}")


5.1 Correccion del campo TECNOLOGIA:
--------------------------------------------------------------------------------
  Estandarizando 'NA (No Aplica)' -> 'NA'
  Total registros con TECNOLOGIA='NA': 166,913


### 5.2 Imputacion de valores nulos

In [25]:
print("\n5.2 Imputacion de valores nulos:")
print("-" * 80)

# Imputar valores facturados con mediana por departamento y servicio
print("\n  VALORES FACTURADOS:")
for campo in ['VALOR_FACTURADO_O_COBRADO', 'OTROS_VALORES_FACTURADOS']:
    nulos_antes = df_analisis[campo].isna().sum()
    if nulos_antes > 0:
        # Calcular mediana por departamento y servicio
        medianas = df_analisis.groupby(['ID_DEPARTAMENTO', 'ID_SERVICIO_PAQUETE'])[campo].median()
        
        # Imputar
        for idx in df_analisis[df_analisis[campo].isna()].index:
            dept = df_analisis.loc[idx, 'ID_DEPARTAMENTO']
            serv = df_analisis.loc[idx, 'ID_SERVICIO_PAQUETE']
            if (dept, serv) in medianas.index:
                valor = int(medianas[(dept, serv)])  # Convertir a int
                df_analisis.at[idx, campo] = valor
            else:
                # Si no hay mediana para esa combinacion, usar 0
                df_analisis.at[idx, campo] = 0
        
        nulos_despues = df_analisis[campo].isna().sum()
        imputados = nulos_antes - nulos_despues
        print(f"    {campo}: {nulos_antes:,} nulos -> {imputados:,} imputados, {nulos_despues:,} restantes")

# Imputar velocidades segun si requieren internet
print("\n  VELOCIDADES:")
servicios_con_internet = [1, 4, 5, 7]

for campo in ['VELOCIDAD_EFECTIVA_DOWNSTREAM', 'VELOCIDAD_EFECTIVA_UPSTREAM']:
    nulos_antes = df_analisis[campo].isna().sum()
    if nulos_antes > 0:
        # Para servicios SIN internet: poner 0
        mask_sin_internet = (~df_analisis['ID_SERVICIO_PAQUETE'].isin(servicios_con_internet)) & (df_analisis[campo].isna())
        df_analisis.loc[mask_sin_internet, campo] = 0.0
        imputados_cero = mask_sin_internet.sum()
        
        # Para servicios CON internet: imputar con mediana por departamento y servicio
        mask_con_internet = (df_analisis['ID_SERVICIO_PAQUETE'].isin(servicios_con_internet)) & (df_analisis[campo].isna())
        medianas = df_analisis[df_analisis['ID_SERVICIO_PAQUETE'].isin(servicios_con_internet)].groupby(['ID_DEPARTAMENTO', 'ID_SERVICIO_PAQUETE'])[campo].median()
        
        for idx in df_analisis[mask_con_internet].index:
            dept = df_analisis.loc[idx, 'ID_DEPARTAMENTO']
            serv = df_analisis.loc[idx, 'ID_SERVICIO_PAQUETE']
            if (dept, serv) in medianas.index:
                df_analisis.at[idx, campo] = float(medianas[(dept, serv)])
            else:
                # Si no hay mediana, usar mediana global del servicio
                mediana_global = df_analisis[df_analisis['ID_SERVICIO_PAQUETE']==serv][campo].median()
                if pd.notna(mediana_global):
                    df_analisis.at[idx, campo] = float(mediana_global)
                else:
                    df_analisis.at[idx, campo] = 0.0
        
        nulos_despues = df_analisis[campo].isna().sum()
        imputados_mediana = nulos_antes - imputados_cero - nulos_despues
        print(f"    {campo}:")
        print(f"      - Servicios sin internet: {imputados_cero:,} -> 0")
        print(f"      - Servicios con internet: {imputados_mediana:,} -> mediana por depto-servicio")
        print(f"      - Nulos restantes: {nulos_despues:,}")


5.2 Imputacion de valores nulos:
--------------------------------------------------------------------------------

  VALORES FACTURADOS:
    VALOR_FACTURADO_O_COBRADO: 1,354 nulos -> 1,354 imputados, 0 restantes
    OTROS_VALORES_FACTURADOS: 17 nulos -> 17 imputados, 0 restantes

  VELOCIDADES:
    VELOCIDAD_EFECTIVA_DOWNSTREAM:
      - Servicios sin internet: 0 -> 0
      - Servicios con internet: 23,768 -> mediana por depto-servicio
      - Nulos restantes: 0
    VELOCIDAD_EFECTIVA_UPSTREAM:
      - Servicios sin internet: 0 -> 0
      - Servicios con internet: 69,974 -> mediana por depto-servicio
      - Nulos restantes: 0


### 5.3 Decision sobre duplicados

In [26]:
print("\n5.3 Decision sobre duplicados:")
print("-" * 80)
print("  DECISION: Mantener duplicados")
print("  JUSTIFICACION: Representan diferentes empresas, tecnologias y velocidades")
print("                 para la misma combinacion municipio-segmento-servicio")

# Dataset limpio final
df_limpio = df_analisis.copy()


5.3 Decision sobre duplicados:
--------------------------------------------------------------------------------
  DECISION: Mantener duplicados
  JUSTIFICACION: Representan diferentes empresas, tecnologias y velocidades
                 para la misma combinacion municipio-segmento-servicio


## RESUMEN FINAL - PARTE 1

In [27]:
print("\n" + "="*80)
print("RESUMEN FINAL - PARTE 1")
print("="*80)

# Verificar nulos finales
nulos_finales = df_limpio.isnull().sum().sum()
print(f"\nDataset limpio: {len(df_limpio):,} registros")
print(f"Campos: {len(df_limpio.columns)}")
print(f"Nulos restantes: {nulos_finales:,}")
print(f"TECNOLOGIA corregida: 'NA (No Aplica)' -> 'NA'")
print(f"Valores facturados imputados con mediana por departamento-servicio")
print(f"Velocidades imputadas segun tipo de servicio")
print(f"Tipos de datos convertidos")
print(f"Listo para Parte 2: Analisis Exploratorio")
print("\nVariable: df_limpio")


RESUMEN FINAL - PARTE 1

Dataset limpio: 1,750,659 registros
Campos: 22
Nulos restantes: 16,582
TECNOLOGIA corregida: 'NA (No Aplica)' -> 'NA'
Valores facturados imputados con mediana por departamento-servicio
Velocidades imputadas segun tipo de servicio
Tipos de datos convertidos
Listo para Parte 2: Analisis Exploratorio

Variable: df_limpio


## EXPORTACION DEL DATASET LIMPIO

In [28]:
# Exportar a CSV
output_filename = 'empaquetamiento_fijo_limpio_2023_2024.csv'
df_limpio.to_csv(output_filename, index=False, encoding='utf-8')
print(f"\nDataset exportado exitosamente a: {output_filename}")


Dataset exportado exitosamente a: empaquetamiento_fijo_limpio_2023_2024.csv
